# 인덱싱·필터링·정제 실전(조회/조건/정렬/클리닝)

In [272]:
import pandas as pd

# 1) 인덱싱이 중요한 이유

인덱싱은 한마디로 말하면 데이터프레임에서 “필요한 부분만 정확히 꺼내는 기술”이다.

전처리와 분석에서 우리가 실제로 반복하는 작업은 항상 다음 세 가지로 수렴한다.

## 1.1) 필요한 행만 가져오기 (Row 선택)
실제 분석에서는 거의 항상 전체 행을 다 보지 않는다.
- 특정 날짜(어제/이번 주)만
- 특정 지점만
- 결제 완료 건만 등
이때 행 선택이 잘못되면(결제 실패 섞임, 기간 섞임 등) 매출/통계가 통째로 왜곡될 수 있다.

## 1.2) 필요한 열만 남기기 (Column 선택)
분석에 모든 컬럼이 필요한 경우는 거의 없다. 떄문에 핵심 컬럼만 남겨 불필요한 컬럼을 지운다.
- 작업이 단순해짐
- 실수(컬럼 혼동)가 줄어듬
- 정제/집계/시각화가 더 빨라짐 (데이터 자체가 축소되기 떄문)

## 1.3) 조건에 맞는 데이터만 보기(Filtering)
분석의 핵심은 비교다. 비교를 하려면 먼저 조건으로 데이터를 나눠야 한다.
- A점 vs B점
- 오전 vs 오후
- 평일 vs 주말
이 조건을 정확히 구해야 비교 분석이 가능해진다

## 1.4) 인덱싱을 잘못 이해하면 생기는 문제
인덱싱이 어설프면 이런 문제가 반복된다.
- 의도하지 않은 행이 섞인다
- 조건이 일부만 적용된다
- 결과는 나오지만 신뢰할 수 없다
인덱싱은 "코드를 치는 기술"이 아닌 분석 조건을 정확히 구현하는 기술이다.


# 2) .loc vs .iloc 핵심 정리 (조회)

## 2.1) 먼저 결론부터 (이것만 먼저 암기)
- .loc → 이름(라벨) 기준으로 고른다
- .iloc → 순서(위치) 기준으로 고른다

이 차이 하나 때문에 슬라이싱 결과가 달라지고, 초보자가 가장 많이 실수한다.

## 2.2) 라벨과 위치의 차이부터 정확히 이해하자

In [273]:
df = pd.DataFrame(
    {"menu": ["Latte", "Americano", "Mocha"],
    "price": [5000, 4500, 5500]},
    index=["A001", "A002", "A003"]
)
# 행의 이름(라벨): "A001", "A002", "A003"
# 행의 순서(위치): 0, 1, 2

## 2.3) .loc : 라벨(이름) 기준 조회
언제 쓰는가?
- 인덱스나 컬럼 이름에 의미가 있을 때
- 조건 필터링과 함께 사용할 때

핵심 특징
- 행 / 열 이름으로 접근
- 문자열 라벨 사용 가능
- 조건 필터링과 궁합이 좋음
- 슬라이싱 시 끝 라벨이 포함될 수 있음

In [274]:
# 이름이 A002인 행을 가져와라
df.loc["A002"]

menu     Americano
price         4500
Name: A002, dtype: object

## 2.4) .iloc : 위치(순서) 기준 조회
언제 쓰는가?
- “위에서 몇 번째”가 중요할 때
- 데이터 구조를 빠르게 확인할 때

핵심 특징
- 정수 위치만 사용
- 파이썬 리스트 슬라이싱과 동일
- 슬라이싱 시 끝은 항상 미포함

## 2.5) 언제 쓰는게 자연스러운가?

In [275]:
df = pd.DataFrame({
    "date":  ["2026-01-01","2026-01-02","2026-01-03","2026-01-04","2026-01-05","2026-01-06","2026-01-07"],
    "store": ["A","A","B","A","B","A","B"],
    "menu":  ["Latte","Americano","Mocha","Latte","Americano","Mocha","Latte"],
    "qty":   [1,2,1,3,1,2,2],
    "paid":  [True, True, False, True, True, False, True]
})
df

,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False
3,2026-01-04,A,Latte,3,True
4,2026-01-05,B,Americano,1,True
5,2026-01-06,A,Mocha,2,False
6,2026-01-07,B,Latte,2,True


### 언제 .loc이 자연스러운가?

#### 1) 조건 필터링이 들어갈 때 (가장 중요)

In [276]:
loc_filtered = df.loc[
    df["paid"] == True,
    ["date", "store", "menu", "qty"]
]
loc_filtered
# paid가 True인 행 중에서 이 컬럼들만 보겠다

,date,store,menu,qty
0,2026-01-01,A,Latte,1
1,2026-01-02,A,Americano,2
3,2026-01-04,A,Latte,3
4,2026-01-05,B,Americano,1
6,2026-01-07,B,Latte,2


#### 2) 인덱스가 의미 있을 때 (날짜 등)

In [277]:
df_dt = df.set_index("date")
df_dt
# 인덱스가 의미 있을 때 (날짜 등)

,store,menu,qty,paid
date,,,,
2026-01-01,A,Latte,1,True
2026-01-02,A,Americano,2,True
2026-01-03,B,Mocha,1,False
2026-01-04,A,Latte,3,True
2026-01-05,B,Americano,1,True
2026-01-06,A,Mocha,2,False
2026-01-07,B,Latte,2,True


In [278]:
# 인덱스가 의미 있을 때 (날짜 등): 범위로 자르기
loc_range = df_dt.loc["2026-01-01":"2026-01-04", ["store", "menu", "qty", "paid"]]
loc_range
# 날짜라는 라벨 범위로 자른다.


,store,menu,qty,paid
date,,,,
2026-01-01,A,Latte,1,True
2026-01-02,A,Americano,2,True
2026-01-03,B,Mocha,1,False
2026-01-04,A,Latte,3,True


### 언제 .iloc이 자연스러운가?

#### 1) 위에서 몇 줄만 보고 싶을 때

In [279]:
df.iloc[0:5]
# 정확히 처음 5행

,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False
3,2026-01-04,A,Latte,3,True
4,2026-01-05,B,Americano,1,True


#### 2) 위치 기준으로 정확히 자를 때

In [280]:
df.iloc[0:10]
# 처음 10개

,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False
3,2026-01-04,A,Latte,3,True
4,2026-01-05,B,Americano,1,True
5,2026-01-06,A,Mocha,2,False
6,2026-01-07,B,Latte,2,True


#### 3) 컬럼 이름이 헷갈릴 때 구조 확인용

In [281]:
df.iloc[:, 0]


0    2026-01-01
1    2026-01-02
2    2026-01-03
3    2026-01-04
4    2026-01-05
5    2026-01-06
6    2026-01-07
Name: date, dtype: object

## 2.6) 슬라이싱 핵심 차이 (끝 포함 여부)
- .loc → 라벨 기준, 끝 포함 가능
- .iloc → 위치 기준, 끝 미포함

In [282]:
df.loc[0:2]    # 0, 1, 2 포함

,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False


In [283]:
df.iloc[0:2]   # 0, 1만 포함

,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True


## 2.7) 가장 많이 하는 실수 (이것만 조심)
“처음 10개만 보자”라는 말에 이렇게 쓰면 위험하다.

In [284]:
df.loc[0:10]   # 11개가 나올 수 있음


,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False
3,2026-01-04,A,Latte,3,True
4,2026-01-05,B,Americano,1,True
5,2026-01-06,A,Mocha,2,False
6,2026-01-07,B,Latte,2,True


In [285]:
df.iloc[0:10]  # 정확히 10개


,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False
3,2026-01-04,A,Latte,3,True
4,2026-01-05,B,Americano,1,True
5,2026-01-06,A,Mocha,2,False
6,2026-01-07,B,Latte,2,True


# 3) 행 / 열 선택 패턴

## 3.1) 왜 이 패턴이 중요한가?
Pandas에서 컬럼을 선택하면 결과가 두 가지 중 하나로 나온다.
- Series (1차원)
- DataFrame (2차원)

문제는 이 둘이 다음 점에서 다르다는 것이다.
- 사용할 수 있는 메서드가 다르다
- 속성(columns, dtypes)이 다르다
- 출력 형태가 다르다


In [286]:
df = pd.DataFrame({
    "menu":  ["Latte", "Americano", "Mocha"],
    "price": [5000, 4500, 5500],
    "paid":  [True, True, False]
})

## 3.2) 단일 컬럼 선택
→ Series가 되는 경우가 많다

In [287]:
menu_series = df["menu"]
menu_series

print("=== menu_series ===")
print(menu_series)

print("\n=== menu_series의 타입 ===")
print(type(menu_series))
print(menu_series.name)
print(menu_series.shape)

=== menu_series ===
0        Latte
1    Americano
2        Mocha
Name: menu, dtype: object

=== menu_series의 타입 ===
<class 'pandas.core.series.Series'>
menu
(3,)


이렇게 컬럼 하나만 선택하면 결과는 보통 Series다.
- 1차원 구조
- 컬럼 이름은 name 속성으로만 유지됨
- columns 속성이 없다

### Series인게 무슨 문제인가?
문제는 다음 줄 코드에서 터진다.

#### 자주 하는 실수 1
```python
df["menu"].columns
```
→ Series에는 columns가 없어서 에러 (columns은 DataFrame의 메소드)


#### 자주 하는 실수 2
```python
df["menu"].merge(other_df, on="menu")
```
→ Series는 merge 흐름에 바로 쓰기 어렵다

Series는 하나의 칼럼을 보거나 계산하기엔 편하지만 작업을 이어가기엔 불안정하다.

## 3.3) 복수 컬럼 선택
컬럼을 리스트로 감싸서 선택하면, 컬럼이 하나여도 DataFrame 형태가 유지된다.

DataFrame의 장점은 명확하다.
- 표 구조 유지
- columns, dtypes 사용 가능
- 정렬 / 저장 / merge / groupby 흐름이 안정적

특히나
- sort_values()
- to_csv()
- merge()
- groupby() 결과 정리
- 리포트용 표 생성

과 같은 다양한 작업들을 막힘 없이 할수 있게 된다.

In [288]:
menu_price_df = df[["menu", "price"]]

menu_df = df[["menu"]] # 이중괄호를 사용해 컬럼이 1개여도 DataFrame로 만들수 있다.

In [289]:
print("=== menu_price_df ===")
print(menu_price_df)

print("\n=== menu_price_df의 타입 ===")
print(type(menu_price_df))
print(menu_price_df.columns)
print(menu_price_df.shape)

print("\n\n=== menu_df ===")
print(menu_df)

print("\n=== menu_df의 타입 ===")
print(type(menu_df))
print(menu_df.columns)
print(menu_df.shape)


=== menu_price_df ===
        menu  price
0      Latte   5000
1  Americano   4500
2      Mocha   5500

=== menu_price_df의 타입 ===
<class 'pandas.core.frame.DataFrame'>
Index(['menu', 'price'], dtype='object')
(3, 2)


=== menu_df ===
        menu
0      Latte
1  Americano
2      Mocha

=== menu_df의 타입 ===
<class 'pandas.core.frame.DataFrame'>
Index(['menu'], dtype='object')
(3, 1)


### 단일 컬럼이라도 DataFrame으로 유지하기
위에서도 설명했으나 이중괄호를 사용하면 컬럼이 1개인 Series도 DataFrame의 형태로 만들수 있다.

```python
menu_df = df[["menu"]]
```
이 코드의 의미는 "컬럼 하나지만 표 형태로 계속 다루겠다"로 읽을수 있다.

## 3.4) “다음 코드가 달라지는” 대표 사례
컬럼을 어떻게 선택했느냐에 따라 결과가 Series가 될 수도 있고 DataFrame이 될 수도 있다.

그렇다면 다음 질문이 나올수도 있다

**그래서 그게 뭐가 문젠데?**


In [290]:
df = pd.DataFrame({
    "date":  ["2026-01-01","2026-01-01","2026-01-02","2026-01-02","2026-01-03"],
    "store": ["A","B","A","B","A"],
    "menu":  ["Latte","Latte","Americano","Mocha","Latte"],
    "price": [5000, 5000, 4500, 5500, 5000],
    "qty":   [1, 2, 1, 1, 3]
})
df

,date,store,menu,price,qty
0,2026-01-01,A,Latte,5000,1
1,2026-01-01,B,Latte,5000,2
2,2026-01-02,A,Americano,4500,1
3,2026-01-02,B,Mocha,5500,1
4,2026-01-03,A,Latte,5000,3


### 결과 모양이 달라져서 join / merge / 저장이 꼬이는 경우

In [291]:
menu_series = df["menu"]     # Series
menu_df     = df[["menu"]]   # DataFrame

겉보기에는 둘 다 "menu 컬럼 하나만 있는 것"처럼 보인다. 하지만 둘의 성질은 완전히 다르다.

##### 1. 저장 관점에서의 차이
- Series
    - 컬럼이라는 개념이 약하다
    - name이 없거나 애매하면 저장 시 헤더가 기대와 다르게 보일 수 있다

- DataFrame
    - columns가 명확히 유지된다
    - CSV/Excel/리포트로 내보내기 안정적이다

##### 2. merge / join 관점에서의 차이
- Series
    - merge()가 기본 흐름이 아니다
    - 바로 쓰려고 하면 에러가 나거나 중간 변환(reset_index, to_frame)이 필요해진다

- DataFrame
    - merge()가 자연스럽게 이어진다
    - 표 + 표 결합이라는 사고 흐름이 유지된다


### 그룹화 / 집계 결과를 이어갈 때

In [292]:
qty_sum_series = df.groupby("menu")["qty"].sum()


위 코드의 대부분의 결과는 Series다.
- index → menu
- values → 합계


##### Series 집계의 불편한 점
Series는 이렇게 생긴 질문에 바로 답하기 어렵다.
- "이걸 리포트용 표로 만들고 싶은데?"
- "menu를 컬럼으로 두고 싶다"
- "다른 집계 결과랑 합치고 싶다"

결국 위 작업을 이어나가기 위해 번거롭게 형변환을 해줘야 한다.
```python
qty_sum_df = qty_sum_series.reset_index(name="total_qty")
```
즉 시작을 Series로 하면 중간에 DataFrame으로 바꾸는 작업이 자주 필요해진다.



In [293]:
qty_price_df = df.groupby("menu").agg(
    total_qty=("qty", "sum"),
    avg_price=("price", "mean")
)

qty_price_df["revenue_est"] = qty_price_df["total_qty"] * qty_price_df["avg_price"]

print(qty_price_df)

           total_qty  avg_price  revenue_est
menu                                        
Americano          1     4500.0       4500.0
Latte              6     5000.0      30000.0
Mocha              1     5500.0       5500.0


처음부터 DataFrame으로 시작하면
- 처음부터 표 형태
- 컬럼 추가가 자연스럽다
- merge / 저장 / 시각화로 바로 이어진다

## 3.5) 초보자에게 추천하는 실무 습관 2가지
앞에서 살펴본 문제들은 대부분 판다스 문법을 몰라서가 아닌, "내가 지금 무엇을 가지고 있는지"를 놓쳐서 생긴다.
때문에 다음 습관들은 반드시 가져보자

In [294]:
df = pd.DataFrame({
    "menu":  ["Latte", "Americano", "Mocha"],
    "price": [5000, 4500, 5500],
    "paid":  [True, True, False]
})
df

,menu,price,paid
0,Latte,5000,True
1,Americano,4500,True
2,Mocha,5500,False


### 습관 A “표 형태를 유지하고 싶으면” 항상 이중 대괄호
아래의 코드 둘 다 menu 컬럼 하나를 고른 것처럼 보이지만,
- df["menu"]
    - 1차원 자료구조 (Series)
    - 다음 단계에서 merge, 저장, 확장이 불편해질 수 있음

- df[["menu"]]
    - 2차원 표 형태 (DataFrame)
    - 이후 작업 흐름이 안정적


In [295]:
df["menu"]     # 보통 Series
df[["menu"]]   # DataFrame

print(type(df["menu"]))
print(type(df[["menu"]]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


그래서 실무에서는 이렇게 생각하는 게 좋다.
- "이 데이터를 나중에 표처럼 다룰 가능성이 있다면 처음부터 DataFrame으로 유지하자."

이를 실천하기 가장 쉬운방법은 이중 대괄호를 습관처럼 사용하는 것이다.

### 습관 B “지금 내가 가진 게 Series인지 DataFrame인지” 한번 확인
에러의 상당수는 아래 상황에서 발생한다.
- "DataFrame이라고 생각하고 코드를 썼는데 사실은 Series였다"

이를 막는 가장 간단한 방법은한 줄만 확인하는 습관이다.

```python
type(변수)
변수.shape
```
- Series
    - shape → (행수,)
    - 열 개념이 없음

- DataFrame
    - shape → (행수, 열수)
    - 표 구조 유지

코드를 길게 분석할 필요 없다. 지금 작업하려는 변수가 어떠한 형태인지만 알고 유의하며 작업에 들어가면 된다.

# 4) 조건 필터링(불리언 인덱싱)의 본질 (중요)
조건 필터링은 "판다스 문법"이 아닌 사고 방식에 가깝다.

In [296]:
df = pd.DataFrame({
    "order_id": [1, 2, 3, 4],
    "menu": ["Latte", "Americano", "Mocha", "Latte"],
    "price": [5000, 4500, 5500, 5000],
    "paid": [True, False, True, False]
})

df

,order_id,menu,price,paid
0,1,Latte,5000,True
1,2,Americano,4500,False
2,3,Mocha,5500,True
3,4,Latte,5000,False


## 4.1) 핵심 아이디어
"True / False 필터(마스크)를 먼저 만든다"

조건 필터링은 항상 2단계로 생각하면 가장 쉽다.
1. 조건을 만족하면 True, 아니면 False인 필터(마스크) 를 만든다
2. 그 필터로 True인 행만 선택한다

예를 들어 "결제가 완료된 주문만 보고 싶다"면,
- 결제 완료 → True
- 결제 실패 → False

이렇게 만든 True/False 배열이 바로 필터(마스크) 다.

어떻게 만드는지 문법 사용 예시

In [297]:
mask = (df["paid"] == True)

paid_orders = df[mask]

print("=== 필터 ===")
print(mask)

print("\n=== 필터 적용 전 ===")
print(df)
print(df.shape)

print("\n=== 필터 적용 후 ===")
print(paid_orders)
print(menu_df.shape)

=== 필터 ===
0     True
1    False
2     True
3    False
Name: paid, dtype: bool

=== 필터 적용 전 ===
   order_id       menu  price   paid
0         1      Latte   5000   True
1         2  Americano   4500  False
2         3      Mocha   5500   True
3         4      Latte   5000  False
(4, 4)

=== 필터 적용 후 ===
   order_id   menu  price  paid
0         1  Latte   5000  True
2         3  Mocha   5500  True
(5, 1)


## 4.2 왜 굳이 True/False 필터를 따로 만들까?
초보자 입장에서는 이렇게 생각하기 쉽다. "바로 df[...]로 뽑으면 되지 않나?"

하지만 실무에서는 3가지 이유 때문에 필터를 분리해두는 쪽이 훨씬 안전하다.
1. 조건에 맞는 행이 몇 개인지 바로 확인 가능
2. 여러 조건을 조합할 때 디버깅이 쉬움
3. 같은 조건을 재사용하기 쉬움

In [298]:
df = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5, 6],
    "store": ["A","A","B","A","B","A"],
    "menu": ["Latte","Americano","Latte","Mocha","Latte","Latte"],
    "price": [5000, 4500, 5000, 5500, 5000, 5000],
    "paid": [True, False, True, True, False, True]
})


In [299]:
# 1. 조건에 맞는 행이 몇 개인지 바로 확인 가능
mask_paid = (df["paid"] == True)
mask_paid.sum()   # True는 1로 계산됨 → 조건에 맞는 행 개수

np.int64(4)

In [300]:
# 2. 여러 조건을 조합할 때 디버깅이 쉬움
mask_storeA = (df["store"] == "A")
mask_latte  = (df["menu"] == "Latte")

mask_final = mask_paid & mask_storeA & mask_latte
mask_final


0     True
1    False
2    False
3    False
4    False
5     True
dtype: bool

In [301]:
# 3. 같은 조건을 재사용하기 쉬움
df[mask_final]
df.loc[mask_final, ["order_id", "menu", "price"]]


,order_id,menu,price
0,1,Latte,5000
5,6,Latte,5000


## 4.3) 다중 조건에서 반드시 지켜야 하는 규칙
조건이 하나일 때는 괜찮지만, 두 개 이상이 되는 순간 실수가 급증한다.

그래서 아래 규칙은 "이해"보다 습관처럼 쓰는 것이 중요하다.

### 규칙 1) &(AND), |(OR), ~(NOT) 사용 시 괄호 필수
괄호를 생략하면 파이썬이 연산 우선순위를 다르게 해석해서 에러 또는 엉뚱한 결과가 나온다.
- AND → (조건1) & (조건2)
- OR → (조건1) | (조건2)
- NOT → ~(조건)

### 규칙 2) and / or 가 아니라 & / | 를 사용한다
- and, or
    - 단일 True/False 판단용
- &, |
    - 판다스의 True/False 배열(필터)용

In [302]:
#-------------------------------
# 사용 예시
#-------------------------------

# AND 조건
mask_and = (df["paid"] == True) & (df["store"] == "A") & (df["menu"] == "Latte")

# OR 조건
mask_or = (df["menu"] == "Latte") | (df["menu"] == "Mocha")

# NOT 조건
mask_not = ~(df["paid"] == True)

print("=== 필터 적용 전 ===")
print(df)

print("\n=== AND 조건 ===")
print(df.loc[mask_and])

print("\n=== OR 조건 ===")
print(df.loc[mask_or])

print("\n=== NOT 조건 ===")
print(df.loc[mask_not])

=== 필터 적용 전 ===
   order_id store       menu  price   paid
0         1     A      Latte   5000   True
1         2     A  Americano   4500  False
2         3     B      Latte   5000   True
3         4     A      Mocha   5500   True
4         5     B      Latte   5000  False
5         6     A      Latte   5000   True

=== AND 조건 ===
   order_id store   menu  price  paid
0         1     A  Latte   5000  True
5         6     A  Latte   5000  True

=== OR 조건 ===
   order_id store   menu  price   paid
0         1     A  Latte   5000   True
2         3     B  Latte   5000   True
3         4     A  Mocha   5500   True
4         5     B  Latte   5000  False
5         6     A  Latte   5000   True

=== NOT 조건 ===
   order_id store       menu  price   paid
1         2     A  Americano   4500  False
4         5     B      Latte   5000  False


## 4.4) 실무에서 가장 안전한 기본 형태

```python
df.loc[조건, 컬럼]
```

조건 필터링의 실무 표준 문장은 이 형태다.
- 조건 → 어떤 행을 고를지
- 컬럼 → 어떤 열만 볼지

In [303]:
df = pd.DataFrame({
    "date": ["2026-01-01","2026-01-01","2026-01-02","2026-01-03"],
    "menu": ["Latte","Americano","Mocha","Latte"],
    "price": [5000, 4500, 5500, 5000],
    "qty": [1, 2, 1, 3],
    "paid": [True, False, True, True]
})

# 1단계 조건 생성
condition = (df["paid"] == True)

# 2단계. 컬럼 정하기 (보여줄 열)
cols = ["date", "menu", "price"]

# 3단계. 기본 형태로 적용: 이 한 줄로 "결제 완료인 행만 + 필요한 컬럼만" 표현 가능
result = df.loc[condition, cols]


print("=== 필터 적용 전 ===")
print(df)


print("\n=== 필터 ===")
print(condition)

print("\n=== 필터 적용 후 ===")
print(result)

=== 필터 적용 전 ===
         date       menu  price  qty   paid
0  2026-01-01      Latte   5000    1   True
1  2026-01-01  Americano   4500    2  False
2  2026-01-02      Mocha   5500    1   True
3  2026-01-03      Latte   5000    3   True

=== 필터 ===
0     True
1    False
2     True
3     True
Name: paid, dtype: bool

=== 필터 적용 후 ===
         date   menu  price
0  2026-01-01  Latte   5000
2  2026-01-02  Mocha   5500
3  2026-01-03  Latte   5000


# 5) 정렬(sort_values, sort_index)이 필요한 이유
정렬은 분석 결과를 사람이 읽기 좋은 순서로 재배치하는 작업이다.

## 5.1) 보고서/리포트는 “정렬된 표”가 기본
실무에서 보고서는 보통 아래 요구를 포함한다.
- 최신순: 최근 데이터부터 보여주기 (날짜 기준 내림차순)
- 매출 큰 순: 매출 상위 메뉴/지점 찾기 (매출 기준 내림차순)
- 수량 많은 순: TOP N 만들기 (수량 기준 내림차순)

정렬이 하는 일은 단순히 보기 좋게 만드는 게 아닌, 우선순위를 표 위에 고정해서 의사결정이 가능하게 만드는 것이다.

### 정렬은 “커뮤니케이션 비용”을 줄인다
분석 결과는 보통 다른 사람에게 보여준다. 정렬이 되어 있으면 보는 사람이 표를 해석하는 과정이 단축된다.
- 무엇이 중요한지 바로 보임
- 질문이 줄어듦
- 보고서 설득력이 올라감

정렬이 안 되어 있으면 반대로 "그래서 결론이 뭔가요?" 같은 질문이 늘어난다.

In [304]:
df = pd.DataFrame({
    "date": ["2026-01-03", "2026-01-01", "2026-01-02", "2026-01-03", "2026-01-01"],
    "menu": ["Latte", "Americano", "Mocha", "Americano", "Latte"],
    "qty":  [2, 1, 3, 1, 1],
    "price":[5000, 4500, 5500, 4500, 5000]
})
df["revenue"] = df["qty"] * df["price"]

In [305]:
latest_first  = df.sort_values(by="date", ascending=False)      # 최신순
revenue_first = df.sort_values(by="revenue", ascending=False)   # 매출 큰 순
top_qty       = df.sort_values(by="qty", ascending=False).head(3)  # 수량 TOP 3

print("=== 최신순 정렬 ===")
print(latest_first)

print("\n=== 매출순 정렬 ===")
print(revenue_first)

print("\n=== 수량 TOP 3 ===")
print(top_qty)

=== 최신순 정렬 ===
         date       menu  qty  price  revenue
0  2026-01-03      Latte    2   5000    10000
3  2026-01-03  Americano    1   4500     4500
2  2026-01-02      Mocha    3   5500    16500
1  2026-01-01  Americano    1   4500     4500
4  2026-01-01      Latte    1   5000     5000

=== 매출순 정렬 ===
         date       menu  qty  price  revenue
2  2026-01-02      Mocha    3   5500    16500
0  2026-01-03      Latte    2   5000    10000
4  2026-01-01      Latte    1   5000     5000
1  2026-01-01  Americano    1   4500     4500
3  2026-01-03  Americano    1   4500     4500

=== 수량 TOP 3 ===
         date       menu  qty  price  revenue
2  2026-01-02      Mocha    3   5500    16500
0  2026-01-03      Latte    2   5000    10000
1  2026-01-01  Americano    1   4500     4500


## 5.2) 정렬을 하면 인사이트가 바로 보인다
집계표는 정렬하지 않으면 이런 문제가 생긴다.
- 1등/꼴등이 누구인지 바로 안 보임
- 중요한 항목과 덜 중요한 항목이 같은 비중으로 섞여 보임
- "뭘 먼저 봐야 하는지"가 표에서 드러나지 않음

반대로 매출 내림차순 같은 정렬을 하면
- 상위 항목이 위로 고정되고
- 하위 항목이 아래로 내려가며
- 비교와 우선순위가 즉시 보인다


In [306]:
df = pd.DataFrame({
    "menu": ["Latte","Americano","Mocha","Latte","Mocha","Americano","Tea","Tea","Latte"],
    "qty":  [2, 1, 1, 3, 2, 4, 5, 1, 1],
    "price":[5000,4500,5500,5000,5500,4500,4000,4000,5000]
})
df["revenue"] = df["qty"] * df["price"]

menu_sales = df.groupby("menu", as_index=False)["revenue"].sum()

menu_sales_unsorted = menu_sales
menu_sales_sorted   = menu_sales.sort_values(by="revenue", ascending=False)

top5     = menu_sales_sorted.head(5)
bottom3  = menu_sales_sorted.tail(3)

menu_sales_unsorted, top5, bottom3


print("=== 메뉴별 집계 ===")
print(menu_sales_unsorted)

print("\n=== 매출 ===")
print(top5)

print("\n=== 매출 하위 ===")
print(bottom3)

=== 메뉴별 집계 ===
        menu  revenue
0  Americano    22500
1      Latte    30000
2      Mocha    16500
3        Tea    24000

=== 매출 ===
        menu  revenue
1      Latte    30000
3        Tea    24000
0  Americano    22500
2      Mocha    16500

=== 매출 하위 ===
        menu  revenue
3        Tea    24000
0  Americano    22500
2      Mocha    16500


## 5.4) sort_values vs sort_index 차이(감 잡기)
정렬은 기준이 두 가지로 갈린다.
- sort_values → "값 기준 정렬"
- sort_index  → "인덱스 기준 정렬"

### sort_values: 값 기준 정렬
매출, 수량, 가격처럼 “컬럼의 값”으로 순위를 만들 때 사용

```python
# 기본 구조
df.sort_values(by="컬럼명", ascending=True)

# 여러 럴럼
df.sort_values(by=["컬럼1", "컬럼2"], ascending=[False, True])
```
- 값(column 값) 기준으로 행 순서를 정렬한다
- 랭킹, 매출순, 수량순 같은 리포트용 정렬에 사용

### sort_index: 인덱스 기준 정렬
날짜를 인덱스로 두었거나, groupby 결과처럼 인덱스가 의미를 가질 때 사용

```python
# 기본 구조
df.sort_index(ascending=True)
```
- 인덱스(행 라벨) 기준으로 순서를 정렬한다
- 날짜 인덱스, groupby 결과 정리에 자주 사용


In [307]:
# 값 기준 정렬: 매출 랭킹표
sales = pd.DataFrame({
    "menu": ["Latte", "Americano", "Mocha", "Tea"],
    "revenue": [32000, 15000, 27000, 9000]
})

sales_sorted_by_value = sales.sort_values(by="revenue", ascending=False)

print("=== 정렬 전 ===")
print(sales)

print("\n=== 정렬 후 ===")
print(sales_sorted_by_value)

=== 정렬 전 ===
        menu  revenue
0      Latte    32000
1  Americano    15000
2      Mocha    27000
3        Tea     9000

=== 정렬 후 ===
        menu  revenue
0      Latte    32000
2      Mocha    27000
1  Americano    15000
3        Tea     9000


In [308]:
# 인덱스 기준 정렬: 날짜를 인덱스로 둔 표
daily = pd.DataFrame({
    "date": ["2026-01-03", "2026-01-01", "2026-01-02"],
    "revenue": [12000, 8000, 15000]
}).set_index("date")
daily_sorted_by_index = daily.sort_index()

print("=== 정렬 전 ===")
print(daily)

print("\n=== 정렬 후 ===")
print(daily_sorted_by_index)


=== 정렬 전 ===
            revenue
date               
2026-01-03    12000
2026-01-01     8000
2026-01-02    15000

=== 정렬 후 ===
            revenue
date               
2026-01-01     8000
2026-01-02    15000
2026-01-03    12000


In [309]:
# groupby 결과 인덱스 정리
orders = pd.DataFrame({
    "menu": ["Latte","Americano","Latte","Mocha","Mocha","Tea"],
    "qty":  [2, 1, 3, 1, 2, 4]
})
qty_sum = orders.groupby("menu")["qty"].sum()
qty_sum_sorted_by_index = qty_sum.sort_index()

print("=== 정렬 전 ===")
print(orders)

print("\n=== 정렬 후 ===")
print(qty_sum_sorted_by_index)


=== 정렬 전 ===
        menu  qty
0      Latte    2
1  Americano    1
2      Latte    3
3      Mocha    1
4      Mocha    2
5        Tea    4

=== 정렬 후 ===
menu
Americano    1
Latte        5
Mocha        3
Tea          4
Name: qty, dtype: int64


# 6) 클리닝(정제)의 4대 문제
클리닝은 “데이터를 예쁘게 만드는 작업”이 아니라 뒤에서 필터링/집계/정렬이 깨지지 않도록, 분석 가능한 형태로 만드는 작업이다.

In [310]:
df = pd.DataFrame({
    " Date ": ["2026-01-01", "2026-01-01", "2026-01-02", None,        "2026-01-02"],
    " Menu ": [" Latte ",   " latte",      "Americano ", "Mocha",     "Americano "],
    " Price ": ["5,000원",   "5,000원",     None,         "5,500원",   None],
    " Qty ":   [1,           1,            2,            None,        2],
    " memo ":  ["test",      "test",       "dup",        "x",         "dup"]  # 분석에 필요 없는 컬럼이라고 가정
})

df 

,Date,Menu,Price,Qty,memo
0,2026-01-01,Latte,"5,000원",1.0,test
1,2026-01-01,latte,"5,000원",1.0,test
2,2026-01-02,Americano,None,2.0,dup
3,None,Mocha,"5,500원",NaN,x
4,2026-01-02,Americano,None,2.0,dup


## 6.1) 컬럼명/구조 문제 (rename, drop)
이 단계의 목표는 한 문장이다.

컬럼 이름을 안정적으로 만들고, 분석에 필요 없는 컬럼을 제거한다.

In [311]:
# 컬럼명 공백 제거 + 소문자 통일: " Date " -> "date"
df.columns = df.columns.str.strip().str.lower()

# 컬럼명 바꾸기: 의미가 더 명확한 이름으로
df = df.rename(columns={"menu": "menu_name"})

# 필요 없는 컬럼 제거: 분석에 안 쓰는 메모 컬럼 같은 것
df = df.drop(columns=["memo"])  

df


,date,menu_name,price,qty
0,2026-01-01,Latte,"5,000원",1.0
1,2026-01-01,latte,"5,000원",1.0
2,2026-01-02,Americano,None,2.0
3,None,Mocha,"5,500원",NaN
4,2026-01-02,Americano,None,2.0


## 6.2) 문자열 문제 (공백/대소문자/불필요 문자)
문자열 문제의 핵심은 “표기 흔들림”을 없애는 것이다

같은 값인데 다르게 저장되어 있으면 그룹핑/필터링/조인이 전부 깨진다.

In [312]:
# menu_name: 앞뒤 공백 제거 + 소문자 통일
df["menu_name"] = df["menu_name"].str.strip().str.lower()

# price: "5,000원" 같은 표현을 숫자로 바꾸기 위한 전처리
df["price"] = (
    df["price"].astype("string")              # None도 안전하게 처리하기 위해
    .str.replace(",", "", regex=False)        # "5,000" -> "5000"
    .str.replace("원", "", regex=False)       # "5000원" -> "5000"
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")  # 변환 실패는 NaN

df

,date,menu_name,price,qty
0,2026-01-01,latte,5000,1.0
1,2026-01-01,latte,5000,1.0
2,2026-01-02,americano,<NA>,2.0
3,None,mocha,5500,NaN
4,2026-01-02,americano,<NA>,2.0


## 6.3) 결측치 문제 (NaN)
초보자 기준 전략 2가지: dropna / fillna

결측치는 무조건 처리해야 하는데, 초보자 단계에서는 선택지가 2개면 충분하다.

- dropna: 신뢰할 수 없는 행은 버린다 (단순)
- fillna: 합리적 값으로 채운다 (실무형)

In [315]:
# dropna: price 또는 qty가 없으면 분석 불가능하다고 보고 제거
df_drop = df.dropna(subset=["price", "qty"])

# fillna: 결측을 "합리적 규칙"으로 채우기
df_fill = df.copy()

# qty: 주문 수량 결측은 1로 가정(예시 규칙)
df_fill["qty"] = df_fill["qty"].fillna(1)

# price: 메뉴별 평균 가격으로 채우기(예시 규칙)
menu_mean_price = df_fill.groupby("menu_name")["price"].transform("mean")
df_fill["price"] = df_fill["price"].fillna(menu_mean_price)


print("=== dropna 후 ===")
print(df_drop)

print("\n=== fillna 후 ===")
print(df_fill)

=== dropna 후 ===
         date menu_name  price  qty
0  2026-01-01     latte   5000  1.0
1  2026-01-01     latte   5000  1.0

=== fillna 후 ===
         date  menu_name  price  qty
0  2026-01-01      latte   5000  1.0
1  2026-01-01      latte   5000  1.0
2  2026-01-02  americano   <NA>  2.0
3        None      mocha   5500  1.0
4  2026-01-02  americano   <NA>  2.0


## 6.4) 중복 문제 (duplicated / drop_duplicates)
중복을 다룰 때는 옵션 2개만 기억하면 된다.
- subset: 무엇을 기준으로 중복인지 판단할지
- keep: first(첫 행 유지) / last(마지막 행 유지)

In [317]:
# 중복 기준 예시: date + menu_name + qty + price가 같으면 같은 주문으로 본다
dup_mask = df_fill.duplicated(
    subset=["date", "menu_name", "qty", "price"],
    keep="first"
)

# 중복 제거(첫 번째 유지)
dedup_first = df_fill.drop_duplicates(
    subset=["date", "menu_name", "qty", "price"],
    keep="first"
)

# 중복 제거(마지막 유지)
dedup_last = df_fill.drop_duplicates(
    subset=["date", "menu_name", "qty", "price"],
    keep="last"
)

print("=== 필터 ===")
print(dup_mask)

print("\n=== 중복 제거(첫 번째 유지) ===")
print(dedup_first)

print("\n=== 중복 제거(마지막 유지) ===")
print(dedup_last)



=== 필터 ===
0    False
1     True
2    False
3    False
4     True
dtype: bool

=== 중복 제거(첫 번째 유지) ===
         date  menu_name  price  qty
0  2026-01-01      latte   5000  1.0
2  2026-01-02  americano   <NA>  2.0
3        None      mocha   5500  1.0

=== 중복 제거(마지막 유지) ===
         date  menu_name  price  qty
1  2026-01-01      latte   5000  1.0
3        None      mocha   5500  1.0
4  2026-01-02  americano   <NA>  2.0
